# Learning FourSquare API with Python

## Introduction

In this lab, you will learn in details how to make calls to the Foursquare API for different purposes. You will learn how to construct a URL to send a request to the API to search for a specific type of venues, to explore a particular venue, to explore a Foursquare user, to explore a geographical location, and to get trending venues around a location. Also, you will learn how to use the visualization library, Folium, to visualize the results.

## Table of Contents

1. <a href="#item1">Foursquare API Search Function</a>
2. <a href="#item2">Explore a Given Venue</a>  
3. <a href="#item3">Explore a User</a>  
4. <a href="#item4">Foursquare API Explore Function</a>  
5. <a href="#item5">Get Trending Venues</a>  

### Import necessary Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: \ 
  - anaconda/osx-64::openssl-1.1.1d-h1de35cc_2
  - defaults/osx-64::openssl-1.1.1d-h1de35ccdone

# All requested packages already installed.

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version

##### Make sure that you have created a Foursquare developer account and have your credentials handy

In [2]:
CLIENT_ID = 'IBFYFDBJUDEPCXTT1LLG3PXQBJNW5HOSGWS1GB0I2MM5COMC' # your Foursquare ID
CLIENT_SECRET = 'PT3YWES5EZX30CBHCEBXUSKCNDC0MBQXCSP4PPMY04BPIFJX' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IBFYFDBJUDEPCXTT1LLG3PXQBJNW5HOSGWS1GB0I2MM5COMC
CLIENT_SECRET:PT3YWES5EZX30CBHCEBXUSKCNDC0MBQXCSP4PPMY04BPIFJX


#### Let's again assume that you are staying at the Conrad hotel. So let's start by converting the Contrad Hotel's address to its latitude and longitude coordinates.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>foursquare_agent</em>, as shown below.

In [3]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7151482 -74.0156573


## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

#### Now, let's assume that it is lunch time, and you are craving Italian food. So, let's define a query to search for Italian food that is within 500 metres from the Conrad Hotel. 

In [4]:
search_query = 'Coffee'
radius = 500
print(search_query + ' .... OK!')

Coffee .... OK!


#### Define the corresponding URL

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IBFYFDBJUDEPCXTT1LLG3PXQBJNW5HOSGWS1GB0I2MM5COMC&client_secret=PT3YWES5EZX30CBHCEBXUSKCNDC0MBQXCSP4PPMY04BPIFJX&ll=40.7151482,-74.0156573&v=20180604&query=Coffee&radius=500&limit=30'

#### Send the GET Request and examine the results

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ee1ca6cede9ed0aedbf0b8a'},
 'response': {'venues': [{'id': '5dba113267232c000803cb37',
    'name': 'For Five Coffee Roasters',
    'location': {'lat': 40.71224,
     'lng': -74.014915,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71224,
       'lng': -74.014915}],
     'distance': 329,
     'postalCode': '10280',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['New York, NY 10280', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1591856194',
    'hasPerk': False},
   {'id': '4f241551e4b01b0c5f0f1361',
    'name': 'WFM Coffee Bar',
    'location': {'address': '270 Greenwich St',
     'c

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.crossStreet,venuePage.id,location.neighborhood
0,5dba113267232c000803cb37,For Five Coffee Roasters,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1591856194,False,40.712240,-74.014915,"[{'label': 'display', 'lat': 40.71224, 'lng': ...",329,10280,US,New York,NY,United States,"[New York, NY 10280, United States]",NaN,NaN,NaN,NaN
1,4f241551e4b01b0c5f0f1361,WFM Coffee Bar,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1591856194,False,40.715542,-74.011436,"[{'label': 'display', 'lat': 40.71554183959961...",358,10007,US,New York,NY,United States,"[270 Greenwich St (Warren St), New York, NY 10...",270 Greenwich St,Warren St,NaN,NaN
2,50a38463e4b0d1371722335f,Hudson/Chambers Coffee Cart,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1591856194,False,40.715638,-74.009613,"[{'label': 'display', 'lat': 40.71563827489798...",512,10007,US,New York,NY,United States,"[Chambers St (at Hudson St), New York, NY 1000...",Chambers St,at Hudson St,NaN,NaN
3,4e4dd91ebd41b76bef941ef0,The Coffee Beanery,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1591856194,False,40.714629,-74.015819,"[{'label': 'display', 'lat': 40.71462886707579...",59,07310,US,Jersey City,NY,United States,"[30 Mall Dr W, Jersey City, NY 07310, United S...",30 Mall Dr W,NaN,NaN,NaN
4,5bbb56cba306190039f416a4,Coffee by La Colombe Coffee Roasters,"[{'id': '5665c7b9498e7d8a4f2c0f06', 'name': 'C...",v-1591856194,False,40.713416,-74.015191,"[{'label': 'display', 'lat': 40.713416, 'lng':...",196,10007,US,New York,NY,United States,"[New York, NY 10007, United States]",NaN,NaN,NaN,NaN


#### Define information of interest and filter dataframe

In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,address,crossStreet,neighborhood,id
0,For Five Coffee Roasters,Coffee Shop,40.712240,-74.014915,"[{'label': 'display', 'lat': 40.71224, 'lng': ...",329,10280,US,New York,NY,United States,"[New York, NY 10280, United States]",NaN,NaN,NaN,5dba113267232c000803cb37
1,WFM Coffee Bar,Coffee Shop,40.715542,-74.011436,"[{'label': 'display', 'lat': 40.71554183959961...",358,10007,US,New York,NY,United States,"[270 Greenwich St (Warren St), New York, NY 10...",270 Greenwich St,Warren St,NaN,4f241551e4b01b0c5f0f1361
2,Hudson/Chambers Coffee Cart,Food Truck,40.715638,-74.009613,"[{'label': 'display', 'lat': 40.71563827489798...",512,10007,US,New York,NY,United States,"[Chambers St (at Hudson St), New York, NY 1000...",Chambers St,at Hudson St,NaN,50a38463e4b0d1371722335f
3,The Coffee Beanery,Coffee Shop,40.714629,-74.015819,"[{'label': 'display', 'lat': 40.71462886707579...",59,07310,US,Jersey City,NY,United States,"[30 Mall Dr W, Jersey City, NY 07310, United S...",30 Mall Dr W,NaN,NaN,4e4dd91ebd41b76bef941ef0
4,Coffee by La Colombe Coffee Roasters,Corporate Coffee Shop,40.713416,-74.015191,"[{'label': 'display', 'lat': 40.713416, 'lng':...",196,10007,US,New York,NY,United States,"[New York, NY 10007, United States]",NaN,NaN,NaN,5bbb56cba306190039f416a4
5,3rd Floor Coffee Bar,Corporate Coffee Shop,40.714757,-74.014329,"[{'label': 'display', 'lat': 40.71475688776351...",120,10282,US,New York,NY,United States,"[200 West St (Murray St), New York, NY 10282, ...",200 West St,Murray St,NaN,58ece658424f93769dfd45d5
6,Laughing Man Coffee & Tea,Coffee Shop,40.714754,-74.017368,"[{'label': 'display', 'lat': 40.71475351814522...",150,10282,US,New York,NY,United States,"[1 N End Ave, New York, NY 10282, United States]",1 N End Ave,NaN,NaN,50202b0ee4b0cabb110c01a2
7,Coffee Cart,Coffee Shop,40.715331,-74.011562,"[{'label': 'display', 'lat': 40.71533065202649...",346,NaN,US,New York,NY,United States,"[Greenwich St & Murray St, New York, NY, Unite...",NaN,Greenwich St & Murray St,NaN,4e6ded5e45dd293273a36c79
8,Coffee Cart (South End & Albany),Breakfast Spot,40.711431,-74.015842,"[{'label': 'display', 'lat': 40.71143109488717...",414,10280,US,New York,NY,United States,"[New York, NY 10280, United States]",NaN,NaN,NaN,502a4fe7e4b0930e7b11b3c0
9,38th floor coffee cart,Coffee Shop,40.714011,-74.013645,"[{'label': 'display', 'lat': 40.71401058638798...",211,10281,US,New York,NY,United States,"[200 Vesey St, New York, NY 10281, United States]",200 Vesey St,NaN,NaN,5214cdc411d2f67e0de859f4


#### Let's visualize the Italian restaurants that are nearby

In [9]:
dataframe_filtered.name

0                 For Five Coffee Roasters
1                           WFM Coffee Bar
2              Hudson/Chambers Coffee Cart
3                       The Coffee Beanery
4     Coffee by La Colombe Coffee Roasters
5                     3rd Floor Coffee Bar
6                Laughing Man Coffee & Tea
7                              Coffee Cart
8         Coffee Cart (South End & Albany)
9                   38th floor coffee cart
10                          Coffee & Lunch
11                 Jack’s Stir Brew Coffee
12             Coffee Cuisine of Church St
13                    Sky Lobby Coffee Bar
14                      Le District Coffee
15               Laughing Man Coffee & Tea
16                    Coffee &  Donut Cart
17                 MBJ Coffee Kiosk @ BMCC
18                            I m a coffee
19                              Joe Coffee
20                   The Bar at Condé Nast
21                               Starbucks
Name: name, dtype: object

In [10]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## 2. Explore a Given Venue
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### A. Let's explore the closest Italian restaurant -- _Harry's Italian Pizza Bar_

In [11]:
venue_id = '5dba113267232c000803cb37' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/5dba113267232c000803cb37?client_id=IBFYFDBJUDEPCXTT1LLG3PXQBJNW5HOSGWS1GB0I2MM5COMC&client_secret=PT3YWES5EZX30CBHCEBXUSKCNDC0MBQXCSP4PPMY04BPIFJX&v=20180604'

#### Send GET request for result

In [12]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'seasonalHours', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto'])


{'id': '5dba113267232c000803cb37',
 'name': 'For Five Coffee Roasters',
 'contact': {},
 'location': {'lat': 40.71224,
  'lng': -74.014915,
  'labeledLatLngs': [{'label': 'display', 'lat': 40.71224, 'lng': -74.014915}],
  'postalCode': '10280',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['New York, NY 10280', 'United States']},
 'canonicalUrl': 'https://foursquare.com/v/for-five-coffee-roasters/5dba113267232c000803cb37',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 7},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
 'likes': {'count': 10,
  'groups': [{'type': 'others', 'count': 10, 'items': []}],
  'summary': '10 Likes'},
 'dislike': False,
 'ok': Fal

### B. Get the venue's overall rating

In [13]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.6


That is not a very good rating. Let's check the rating of the second closest Italian restaurant.

In [14]:
venue_id = '52a3a47e11d2b5d5943d33dc' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Since this restaurant has no ratings, let's check the third restaurant.

In [15]:
venue_id = '597a11ccb2958f056b27c60d' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.8


Since this restaurant has a slightly better rating, let's explore it further.

### C. Get the number of tips

In [16]:
result['response']['venue']['tips']['count']

2

### D. Get the venue's tips
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**

#### Create URL and send GET request. Make sure to set limit to get all tips

In [17]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ee1cc42be946f5f08516642'},
 'response': {'tips': {'count': 2,
   'items': [{'id': '5d1a223e4f9e9600238ad558',
     'createdAt': 1561993790,
     'text': 'Cold brew and matcha donuts',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5d1a223e4f9e9600238ad558',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 2,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '53971514',
      'firstName': 'Dan',
      'lastName': 'E',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/53971514-WICEDBVSEOXXYGRC.jpg'}},
     'authorInteractionType': 'liked'}]}}}

#### Get tips and list of associated features

In [18]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

#### Format column width and display all tips

In [19]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

/Users/manjunathvhatkar/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,Cold brew and matcha donuts,2,0,5d1a223e4f9e9600238ad558,Dan,E,NaN,53971514


Now remember that because we are using a personal developer account, then we can access only 2 of the restaurant's tips, instead of all 15 tips.

## 3. Search a Foursquare User
> `https://api.foursquare.com/v2/users/`**USER_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### Define URL, send GET request and display features associated with user

In [46]:
user_id ='53971514' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

KeyError: 'user'

In [ ]:
print('First Name: ' + user_data['firstName'])
print('Last Name: ' + user_data['lastName'])
print('Home City: ' + user_data['homeCity'])

#### How many tips has this user submitted?

In [24]:
user_data['tips']

NameError: name 'user_data' is not defined

Wow! So it turns out that Nick is a very active Foursquare user, with more than 250 tips.

### Get User's tips

In [25]:
# define tips URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,This place is the best in town for breakfast and coffee. Get the burrito or a biscuit sandwich,0,0,5e480f399b9e0c0007339a3b
1,"This is the real Joe’s, and it’s the best. Not quite the same as the Carmine location but unbeatable in the area, and they take cards",0,0,5e1bbccafd56440008d643e4
2,HOLD THAT TIGER,0,0,5e1bbc6fa259900007a6ce6e
3,"Lots of great selections, including natural wines",0,0,5e1bbab5e5465d0006fc8107
4,Get a bottle of wine next store at their wine shop to enjoy with your tapas,0,0,5e1bba9a821fbb00067870b8
5,Everything here is great. Don’t overthink it and don’t ask for suggestions or substitutions,0,0,5e1bb9a8fa9296000aa0e29e
6,"This is Poulet Sans Tête. All the sides and chicken options are excellent, and delivery is reliable. Pulled means de-boned.",0,0,5e1bb95cfd56440008d2af43
7,"Pernil, and lots of hot sauce",0,0,5dfa72c7ac131a00066f5399
8,"Dope wine bar with good snacks. Read about the wines you’re going to try, and get the flights to try a variety.",0,0,5dfa729b36f1760006df7cf9
9,"Delightful doughnuts. Grab one of their whatever is special, and of course a classic like the glazed.",0,0,5dfa725114981a000607c600


In [26]:
tip_id = '5ab5575d73fe2516ad8f363b' # tip id

# define URL
url = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])

Cowgirl
{'address': '519 Hudson St', 'crossStreet': 'at W 10th St', 'lat': 40.73373338282062, 'lng': -74.0062998849649, 'labeledLatLngs': [{'label': 'display', 'lat': 40.73373338282062, 'lng': -74.0062998849649}], 'postalCode': '10014', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['519 Hudson St (at W 10th St)', 'New York, NY 10014', 'United States']}


### Get User's friends

In [27]:
user_friends = json_normalize(user_data['friends']['groups'][0]['items'])
user_friends

NameError: name 'user_data' is not defined

Interesting. Despite being very active, it turns out that Nick does not have any friends on Foursquare. This might definitely change in the future.

### Retrieve the User's Profile Image

In [28]:
user_data

NameError: name 'user_data' is not defined

In [29]:
# 1. grab prefix of photo
# 2. grab suffix of photo
# 3. concatenate them using the image size  
Image(url='https://igx.4sqi.net/img/user/300x300/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg')

## 4. Explore a location
> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**

#### So, you just finished your gourmet dish at Ecco, and are just curious about the popular spots around the restaurant. In order to explore the area, let's start by getting the latitude and longitude values of Ecco Restaurant.

In [30]:
latitude = 40.715337
longitude = -74.008848

#### Define URL

In [31]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=IBFYFDBJUDEPCXTT1LLG3PXQBJNW5HOSGWS1GB0I2MM5COMC&client_secret=PT3YWES5EZX30CBHCEBXUSKCNDC0MBQXCSP4PPMY04BPIFJX&ll=40.715337,-74.008848&v=20180604&radius=500&limit=30'

#### Send GET request and examine results

In [32]:
import requests

In [33]:
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around Ecco restaurant.'

In [34]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '54148bc6498ea7bb8c05b70a',
  'name': 'Juice Press',
  'location': {'address': '83 Murray St',
   'crossStreet': 'btwn Greenwich St & W Broadway',
   'lat': 40.71478769908051,
   'lng': -74.0111317502157,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.71478769908051,
     'lng': -74.0111317502157}],
   'distance': 202,
   'postalCode': '10007',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['83 Murray St (btwn Greenwich St & W Broadway)',
    'New York, NY 10007',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1d3941735',
    'name': 'Vegetarian / Vegan Restaurant',
    'pluralName': 'Vegetarian / Vegan Restaurants',
    'shortName': 'Vegetarian / Vegan',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
  

In [35]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,btwn Greenwich St & W Broadway,40.714788,-74.011132,"[{'label': 'display', 'lat': 40.71478769908051, 'lng': -74.0111317502157}]",202,10007,US,New York,NY,United States,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]",NaN,54148bc6498ea7bb8c05b70a
1,Korin,Furniture / Home Store,57 Warren St,Church St,40.714824,-74.009404,"[{'label': 'display', 'lat': 40.71482437714839, 'lng': -74.00940425461492}, {'label': 'entrance', 'lat': 40.714727, 'lng': -74.009399}]",73,10007,US,New York,NY,United States,"[57 Warren St (Church St), New York, NY 10007, United States]",Tribeca,4af5d65ff964a52091fd21e3
2,Los Tacos No. 1,Taco Place,136 Church St,NaN,40.714267,-74.008756,"[{'label': 'display', 'lat': 40.714267, 'lng': -74.008756}]",119,10007,US,New York,NY,United States,"[136 Church St, New York, NY 10007, United States]",NaN,5d5f24ec09484500079aee00
3,Takahachi Bakery,Bakery,25 Murray St,at Church St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.713652845301894, 'lng': -74.0088038953017}, {'label': 'entrance', 'lat': 40.713716, 'lng': -74.008443}]",187,10007,US,New York,NY,United States,"[25 Murray St (at Church St), New York, NY 10007, United States]",NaN,4c154c9a77cea593c401d260
4,Takahachi,Sushi Restaurant,145 Duane St,btwn W Broadway & Church St,40.716526,-74.008101,"[{'label': 'display', 'lat': 40.71652647412374, 'lng': -74.00810108466207}, {'label': 'entrance', 'lat': 40.716508, 'lng': -74.007989}]",146,10013,US,New York,NY,United States,"[145 Duane St (btwn W Broadway & Church St), New York, NY 10013, United States]",NaN,4a8f2f39f964a520471420e3
5,Exceed Physical Culture,Gym / Fitness Center,97 Reade St,bet W Broadway & Church St,40.715629,-74.007992,"[{'label': 'display', 'lat': 40.7156286200256, 'lng': -74.0079922583853}, {'label': 'entrance', 'lat': 40.715589, 'lng': -74.008105}]",79,10013,US,New York,NY,United States,"[97 Reade St (bet W Broadway & Church St), New York, NY 10013, United States]",Tribeca,53910ac3498e57a5dc0eb160
6,Equinox Tribeca,Gym,54 Murray St,at W Broadway,40.714099,-74.009686,"[{'label': 'display', 'lat': 40.71409860726041, 'lng': -74.0096857179283}]",154,10007,US,New York,NY,United States,"[54 Murray St (at W Broadway), New York, NY 10007, United States]",NaN,4a6e331af964a52031d41fe3
7,Whole Foods Market,Grocery Store,270 Greenwich Street,at Warren St,40.715579,-74.011368,"[{'label': 'display', 'lat': 40.715579155420606, 'lng': -74.01136823958119}]",214,10007,US,New York,NY,United States,"[270 Greenwich Street (at Warren St), New York, NY 10007, United States]",Tribeca,49bc3b0af964a52020541fe3
8,Chambers Street Wines,Wine Shop,148 Chambers St,btwn West Broadway & Hudson St,40.715773,-74.009718,"[{'label': 'display', 'lat': 40.715773063928374, 'lng': -74.00971823312332}, {'label': 'entrance', 'lat': 40.715696, 'lng': -74.00988}]",88,10007,US,New York,NY,United States,"[148 Chambers St (btwn West Broadway & Hudson St), New York, NY 10007, United States]",NaN,4adcf23cf964a520cc6221e3
9,Nish Nūsh,Falafel Restaurant,88 Reade St,at Church St,40.715537,-74.007725,"[{'label': 'display', 'lat': 40.71553710116416, 'lng': -74.00772452925565}, {'label': 'entrance', 'lat': 40.715615, 'lng': -74.00773}]",97,10013,US,New York,NY,United States,"[88 Reade St (at Church St), New York, NY 10013, United States]",NaN,50ba9119e4b071a4bae6dc10


In [36]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

## 5. Explore Trending Venues
> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**

In [37]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ee1cd3165fdfb234d9af771'},
 'response': {'venues': []}}

### Check if any venues are trending at this time

In [38]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [39]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

### Visualize trending venues

In [40]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [41]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'

### Thank you for completing this lab!